# Implement seq2seq
---

## 1. 필요한 모듈을 불러온다.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from pprint import pprint
from tools import *
from tqdm import tqdm

tf.reset_default_graph() # 텐서플로우 그래프를 초기화 한다.

/home/caitech/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 2. corpus(YOLO의 output)를 불러온다.

In [18]:
# pandas DataFrame을 numpy.ndarray로 바꾸려면 '.values'만 추가하면 된다.
# => pd.read_table(...).values
corpus = pd.read_table("./raw.csv", delimiter=",").values

print((corpus, type(corpus)))
print("\ncorpus's shape: {}".format(corpus.shape))

(array([['dallas take off', ' plane take off from dallas'],
       ['dallas land', ' plane land at Dallas'],
       ['chicago take off', ' plane take off from chicago'],
       ['chicago land', ' plane land at chicago'],
       ['los angeles take off', ' plane take off from los angeles'],
       ['los angeles land', ' plane land at los angeles'],
       ['new york take off', ' plane take off from new york'],
       ['new york land', ' plane land at new york'],
       ['miami take off', ' plane take off from miami'],
       ['miami land', ' plane land at miami']], dtype=object), <class 'numpy.ndarray'>)

corpus's shape: (10, 2)


## 3. lookup table를 생성하고 이를 이용하여 데이터를 인덱스로 맵핑한다.

- seq2seq 모델의 cell은 인풋값으로 index로 된 numpy array나 리스트를 받는다.
- tools.py의 make_vocab함수로 먼저 lookup table을 생성하고

In [3]:
idx2word, word2idx, vocab_size = make_vocab(corpus)

temp_encoder_data, temp_decoder_data, temp_targets_data = make_temp_data(corpus,idx2word=idx2word, word2idx=word2idx)

print('vocab_size: {} \n'.format(vocab_size))
print('idx2word: \n {} \n'.format(idx2word))
print('word2idx: \n {} \n'.format(word2idx))

print('temp_encoder_data: \n {} \n'.format(temp_encoder_data))
print('temp_decoder_data: \n {} \n'.format(temp_decoder_data))
print('temp_targetsr_data: \n {} \n'.format(temp_targets_data))

vocab_size: 17 

idx2word: 
 {0: 'from', 1: 'miami', 2: 'los', 3: 'chicago', 4: 'off', 5: 'take', 6: 'angeles', 7: 'plane', 8: 'new', 9: 'land', 10: 'dallas', 11: 'york', 12: 'at', 13: '<start>', 14: '<end>', 15: '<pad>', 16: '<unk>'} 

word2idx: 
 {'from': 0, 'miami': 1, 'los': 2, 'chicago': 3, 'off': 4, 'take': 5, 'angeles': 6, 'plane': 7, 'new': 8, 'land': 9, 'dallas': 10, 'york': 11, 'at': 12, '<start>': 13, '<end>': 14, '<pad>': 15, '<unk>': 16} 

temp_encoder_data: 
 [[10, 5, 4], [10, 9], [3, 5, 4], [3, 9], [2, 6, 5, 4], [2, 6, 9], [8, 11, 5, 4], [8, 11, 9], [1, 5, 4], [1, 9]] 

temp_decoder_data: 
 [[7, 5, 4, 0, 10], [7, 9, 12, 10], [7, 5, 4, 0, 3], [7, 9, 12, 3], [7, 5, 4, 0, 2, 6], [7, 9, 12, 2, 6], [7, 5, 4, 0, 8, 11], [7, 9, 12, 8, 11], [7, 5, 4, 0, 1], [7, 9, 12, 1]] 

temp_targetsr_data: 
 [[7, 5, 4, 0, 10], [7, 9, 12, 10], [7, 5, 4, 0, 3], [7, 9, 12, 3], [7, 5, 4, 0, 2, 6], [7, 9, 12, 2, 6], [7, 5, 4, 0, 8, 11], [7, 9, 12, 8, 11], [7, 5, 4, 0, 1], [7, 9, 12, 1]] 



## 4. 각 데이터에 pad, end 토큰을 삽입하여 최종적인 데이터를 만든다.

In [4]:
encoder_data, decoder_data, targets_data, max_decoder_steps = insert_tokens(temp_encoder_data, temp_decoder_data, temp_targets_data, word2idx)

data_checker(encoder_data, idx2word)
data_checker(decoder_data, idx2word)
data_checker(targets_data, idx2word)

dallas take off <pad>
dallas land <pad> <pad>
chicago take off <pad>
chicago land <pad> <pad>
los angeles take off
los angeles land <pad>
new york take off
new york land <pad>
miami take off <pad>
miami land <pad> <pad>
<start> plane take off from dallas <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane land at dallas <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane take off from chicago <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane land at chicago <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane take off from los angeles <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane land at los angeles <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane take off from new york <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane land at new york <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<start> plane take off from miami <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

## 5. mask를 사용하기 위해서 sequence_length를 구한다.

In [5]:
encoder_sequence_length = sequence_length_maker(encoder_data, word2idx)
decoder_sequence_length = sequence_length_maker(decoder_data, word2idx)
print(encoder_sequence_length)
print(decoder_sequence_length)

[3, 2, 3, 2, 4, 3, 4, 3, 3, 2]
[6, 5, 6, 5, 7, 6, 7, 6, 6, 5]


## 6. 하이퍼파라미터를 정의한다.

버퍼사이즈 레퍼런스
https://stackoverflow.com/questions/46444018/meaning-of-buffer-size-in-dataset-map-dataset-prefetch-and-dataset-shuffle

In [6]:
## 3. 하이퍼파라미터를 설정한다.
class_size = vocab_size
batch_size = 3
lr = 0.01
hidden_size = 128
epochs = 100
save_per_ckpt = 10
num_layer = 1 # stacked RNN을 위한 하이퍼파라미터. 몇개의 레이어를 쌓을지 정해준다.

## 7. 모델을 클래스로 정의한다.

In [7]:
## seq2seq 모델을 만든다.

class Object2Text:
    '''
    Arguments:
        btc_enc_seq_len: placeholder([None,]), 인코더 부분의 패딩을 제외한 실제 의미를 가지는 토큰의 개수를 배치별로 나타냄.
        btc_enc_inp: placeholder([None, None]), 인코더의 인풋(batch size, time steps)
        btc_dec_seq_len: placeholder([None,]), 디코더 부분의 패딩을 제외한 실제 의미를 가지는 토큰의 개수를 배치별로 나타냄.
        btc_dec_inp: placeholder([Noen, None]), 디코더의 인풋(batch size, time steps)
        btc_targets: placeholder([None, None]), groud truth(batch size, time steps)
        word2idx: dictionary, '단어'를 키 값으로 조회하면 그에 해당하는 인덱스를 돌려주는 딕셔너리.
        idx2word: dictionary, '인덱스'를 키 값으로 조회하면 그에 해당하는 인덱스를 돌려주는 딕셔너리.
        vocab_size: scalar, 단어사전의 크기(단어사전에 있는 단어의 개수)
        class_size: scalar, 디코더의 아웃풋을 fully connected 레이어와 연결하여 최종적으로 몇개의 클래스로 나타낼지 결정. 일반적으로 vocab size와 같게 하면 된다.
        hidden_size: scalar, seq2seq 모델의 히든 유닛의 개수
    '''
    
    def __init__(self, btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets, word2idx=word2idx, idx2word=idx2word, 
                     vocab_size=vocab_size, class_size=class_size, hidden_size=hidden_size, num_layer=num_layer, max_decoder_steps=max_decoder_steps):
        
        # 생성자의 인자들을 클래스멤버(?)로 만들어준다.
        self._enc_seq_len = btc_enc_seq_len
        self._enc_inp = btc_enc_inp
        self._dec_seq_len = btc_dec_seq_len
        self._dec_inp = btc_dec_inp
        self._targets = btc_targets
        self._word2idx = word2idx
        self._idx2word = idx2word
        self._vocab_size = vocab_size
        self._class_size = class_size
        self._hidden_size = hidden_size
        self._num_layer = num_layer
        self._max_decoder_steps = max_decoder_steps
    
        # 1) encoder의 인풋값을 ont-hot 인코딩(임베딩) 한다.
        with tf.variable_scope('enc_embed_layer'):
            encoder_oh = tf.one_hot(indices=self._enc_inp, depth=self._vocab_size) # encoder_oh: tensor, (batch_size, time steps, vocab_size)

        #  2) encoder 셀을 쌓는다.
        with tf.variable_scope('encoder'):
            enc_cell = tf.contrib.rnn.BasicLSTMCell(num_units=self._hidden_size, activation=tf.nn.tanh)
            enc_cell = tf.nn.rnn_cell.MultiRNNCell([enc_cell] * self._num_layer)
            _outputs, enc_states = tf.nn.dynamic_rnn(cell=enc_cell, inputs=encoder_oh, sequence_length=self._enc_seq_len, dtype=tf.float32)

        # 3) 디코더의 임베딩을 원-핫인코딩으로 한다. tf.one_hot 함수를 사용해서 원-핫 인코딩을 해도 되지만. 나중에 GreedyEmbeddingsHelper를 사용하기 위해서는 임베딩 매트릭스를 만들어 주어야 한다.
        with tf.variable_scope('dec_embed_layer'):
            decoder_eye = tf.eye(num_rows=self._vocab_size)
            decoder_embeddings = tf.get_variable(name='dec_embed', initializer=decoder_eye, trainable=False) # tf.nn.embedding_lookup 함수를 사용하려면 tf.get_variable을 params의 값으로

            #decoder_embeddings = tf.get_variable(name='dec_embed', initializer=decoder_eye, trainable=False) # tf.nn.embedding_lookup 함수를 사용하려면 tf.get_variable을 params의 값으로
            decoder_oh = tf.nn.embedding_lookup(params=decoder_embeddings, ids=self._dec_inp)    

        # 4) decoder 셀을 쌓는다.    
        with tf.variable_scope('decoder'):
            
            # decoder cell for both training and inference(학습과 추론과정은 같은 '가중치'를 공유함.)
            dec_cell = tf.contrib.rnn.BasicLSTMCell(self._hidden_size, activation=tf.nn.tanh)
            dec_cell = tf.nn.rnn_cell.MultiRNNCell([dec_cell] * self._num_layer)
            
            # output projection (replacing `OutputProjectionWrapper`)
            output_layer = tf.layers.Dense(vocab_size)
#             score_cell = tf.contrib.rnn.OutputProjectionWrapper(cell=dec_cell, output_size=self._class_size)

            # 4-1) decoder의 training과 관련된 셀을 쌓는다.
            with tf.variable_scope('training'):
                train_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_oh, sequence_length=self._dec_seq_len, time_major=False)
                train_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell, helper=train_helper, initial_state=enc_states, output_layer=output_layer)
                
                # self._train_outputs.rnn_output: [batch_size x max(self._max_decoder_steps) x vocab_size]
                # => 각 배치에서 가장 '긴' 값을 2번째 차원의 값으로 가진다는 점을 주의!! seq2seq2 loss 함수를 사용할 때 shape에 주의해야 한다.
                # => bucketing과 연관이 있는 것 같은데.... 확실하지는 않다...
                self._train_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder=train_decoder, impute_finished=True, output_time_major=False, 
                                                                                                                maximum_iterations=self._max_decoder_steps)
                
                # logits: [batch_size x max_dec_step x dec_vocab_size]
                # => tf.identity: seq2seq 함수에서 logits에 들어갈 값 선언. tf.identity는 tensor의 contents와 shape를 똑같이 copy한다.
                logits = tf.identity(self._train_outputs.rnn_output, name='logits')
                
                # btc_max_len: => logits의 shape에 맞춰주기 위함!!
                # => decoder input "배치" 중에서  최대 sequence length(의미 있는 인풋이 들어오는 time step)
                btc_max_len = tf.reduce_max(self._dec_seq_len, name='max_dec_len')
                
                # targets: 
                targets = tf.slice(input_=self._targets, begin=[0, 0], size=[-1, btc_max_len], name='targets')
                              

            # 4-2) decoder의 inference와 관련된 셀을 쌓는다.
            with tf.variable_scope('inference'):
                # 모델의 input을 dynamic shape로 하였으므로.. tf.shape(btc_enc_seq_len)로 해야 한다. btc_enc_seq_len.shape는 static shape일 때 사용 하는 것이다.
                start_tokens = tf.fill(dims=tf.shape(btc_enc_seq_len), value=self._word2idx['<start>'])

                infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding=decoder_embeddings, 
                                                                                                              start_tokens=start_tokens, 
                                                                                                              end_token=self._word2idx['<end>'])
                infer_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell, helper=infer_helper, initial_state=enc_states, output_layer=output_layer)
                self._infer_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(infer_decoder, impute_finished=True, maximum_iterations=self._max_decoder_steps)
        
        with tf.variable_scope('seq2seq_loss'):
            #masksing: [batch_size x max_dec_len]
            #=> ignore outputs after `dec_senquence_length+1` when calculating loss
            masking = tf.sequence_mask(lengths=self._dec_seq_len, maxlen=btc_max_len, dtype=tf.float32)
                
            #internal: `tf.nn.sparse_softmax_cross_entropy_with_logits`
            self.seq2seq_loss = tf.contrib.seq2seq.sequence_loss(logits=logits, targets=targets, weights=masking)
            
    def infer(self, sess,  btc_enc_seq_len, btc_enc_inp):
        feed_infer = {self._enc_seq_len: btc_enc_seq_len, self._enc_inp: btc_enc_inp}
        return sess.run(self._infer_outputs.sample_id, feed_dict=feed_infer)            

## 8. 배치 관련 설정을 한다.

In [8]:
## 인풋, 타겟, sequence_length와 관련된 placeholder를 정의한다.
encoder_input = tf.placeholder(dtype=tf.int32, shape=[None, None]) # (batch size, time steps)
decoder_input = tf.placeholder(tf.int32, [None, None]) # (batch size, time steps)
targets = tf.placeholder(tf.int32, [None, None]) # (batch size, time steps)
encoder_seq_len = tf.placeholder(tf.int32, [None]) # (batch size, ) 인코더 배치에 대응하는 sequence_length
decoder_seq_len = tf.placeholder(tf.int32, [None]) # (batch size,) 디코더 배치에 대응하는 sequence_length

## 배치 iterator를 정의하여 학습시 사용할 준비를 한다.
dataset = tf.data.Dataset.from_tensor_slices((encoder_seq_len, encoder_input, decoder_seq_len, decoder_input, targets)) # 여러 플레이스 홀더를 데이터셋으로 하고
dataset = dataset.shuffle(10000).batch(batch_size) # 랜덤으로 데이터셋을 섞은 다음에 배치크기는 미리 정의된 만큼 할 생각이야.
iterator = dataset.make_initializable_iterator() # 위와 같은 사항을 반영하여 전체 데이터셋에서 배치 크기만큼 반복적으로 꺼내올 수 있는 iterator를 만들어 줘.
btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets  = iterator.get_next() # 배치를 꺼내오는 명령문은 next_batch로 할게. (sess.run에서 실행 시켜야 다음 배치를 꺼내온다.)
# next_element = iterator.get_next() # 배치를 꺼내오는 명령문은 next_batch로 할게. (sess.run에서 실행 시켜야 다음 배치를 꺼내온다.)

print(dataset)
# print(next_element)
print(btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets)

<BatchDataset shapes: ((?,), (?, ?), (?,), (?, ?), (?, ?)), types: (tf.int32, tf.int32, tf.int32, tf.int32, tf.int32)>
Tensor("IteratorGetNext:0", shape=(?,), dtype=int32) Tensor("IteratorGetNext:1", shape=(?, ?), dtype=int32) Tensor("IteratorGetNext:2", shape=(?,), dtype=int32) Tensor("IteratorGetNext:3", shape=(?, ?), dtype=int32) Tensor("IteratorGetNext:4", shape=(?, ?), dtype=int32)


## 9. object2text 클래스를 생성한다.

In [9]:
object2text = Object2Text(btc_enc_seq_len, btc_enc_inp, btc_dec_seq_len, btc_dec_inp, btc_targets)

## 10. optimizer를 정의하고 모델을 학습시킨다.

In [10]:
train_op = tf.train.AdamOptimizer(lr).minimize(object2text.seq2seq_loss)

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
loss_hist=[]

for epoch in tqdm(range(100)):
    
    # iterator.initalizer를 반복문 안에 넣어줘야 배치를 다 꺼내 쓴 다음에 다시 배치를 쓸 수 있다. 이것을 반복문 바깥에 두면 계속 OutOfRangeError 에러가 난다.
    sess.run(iterator.initializer, feed_dict = {encoder_input: encoder_data, decoder_input: decoder_data, 
                                                targets: targets_data, encoder_seq_len: encoder_sequence_length, decoder_seq_len: decoder_sequence_length})
    print("Epoch: %s" %epoch)

    while True:
        try:
            _, loss = sess.run([train_op, object2text.seq2seq_loss])
            print(loss)            
            
        # 에러가 발생한다는 것은 더 이상 꺼내 올 배치가 없다는 것이다. 따라서 배치를 꺼내오는 while문을 종료하고 다음 epoch로 넘어간다.
        except tf.errors.OutOfRangeError:
            break

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0


  6%|▌         | 6/100 [00:00<00:07, 12.51it/s]

2.8287358
2.7516203
2.631255
2.3359888
Epoch: 1
2.556924
2.2986786
2.195085
2.2393222
Epoch: 2
2.0021868
1.8802986
1.7426249
1.5553432
Epoch: 3
1.4535738
1.3574306
1.3032799
0.77499694
Epoch: 4
1.1055855
1.0190586
0.97734785
0.6990683
Epoch: 5
0.76777536
0.76025045
0.658766
0.7378437
Epoch: 6
0.5551729
0.5099685
0.7153764
0.69321495
Epoch: 7
0.50605214
0.480077
0.5381617
0.5194366
Epoch: 8
0.42977202
0.41830173
0.59515595
0.44226918
Epoch: 9


 16%|█▌        | 16/100 [00:00<00:03, 22.51it/s]

...

0.001608737
0.0010912016
Epoch: 80
0.0013994945
0.0014597302
0.0015136638
0.0012797404
Epoch: 81
0.0014498163
0.0013333339
0.0014304694
0.0013914515
Epoch: 82
0.0013044509
0.0014076023
0.00140154
0.001360531
Epoch: 83
0.001205643
0.0012447758
0.0015661964
0.0012649634
Epoch: 84
0.0014508312
0.0011914894
0.0012278692
0.0013165859
Epoch: 85
0.0012486491
0.0012988743
0.0011075006
0.0016518434
Epoch: 86
0.0013555664
0.0011479827
0.0012359077
0.0011294759
Epoch: 87
0.0010351454
0.0012422802
0.0013791865
0.001087058
Epoch: 88
0.0011842374
0.0012356053
0.001207254


 95%|█████████▌| 95/100 [00:02<00:00, 37.72it/s]

0.000894703
Epoch: 89
0.0010939206
0.0011869469
0.0010726748
0.0014667008
Epoch: 90
0.0011192248
0.0011360352
0.0010307959
0.0014110586
Epoch: 91
0.0012450226
0.0011255438
0.0010372499
0.00081452134
Epoch: 92
0.0010338356
0.001123039
0.0010743053
0.0011081869
Epoch: 93
0.0011755385
0.0009302222
0.0010919616
0.0009621973
Epoch: 94
0.001166193
0.0009117643
0.0009737624
0.0011640359
Epoch: 95
0.0010410668
0.0010441467
0.00095650007
0.0010310569
Epoch: 96
0.0010060468
0.00094787957
0.0010479729
0.00090601266
Epoch: 97
0.0010003975
0.0009573049
0.0009649473


100%|██████████| 100/100 [00:02<00:00, 37.97it/s]

0.0009888591
Epoch: 98
0.000882193
0.0009075213
0.0010204638
0.0011050627
Epoch: 99
0.00081811537
0.0009629692
0.0009432935
0.001120022


## 11. 학습된 가중치가 저장된 sess를 통해서 inference를 한다.

In [12]:
prediction = object2text.infer(sess, btc_enc_inp=encoder_data, btc_enc_seq_len=encoder_sequence_length)

In [13]:
print(prediction)

[[ 7  5  4  0 10 14  0]
 [ 7  9 12 10 14  0  0]
 [ 7  5  4  0  3 14  0]
 [ 7  9 12  3 14  0  0]
 [ 7  5  4  0  2  6 14]
 [ 7  9 12  2  6 14  0]
 [ 7  5  4  0  8 11 14]
 [ 7  9 12  8 11 14  0]
 [ 7  5  4  0  1 14  0]
 [ 7  9 12  1 14  0  0]]


In [14]:
list(map(lambda row: [idx2word[idx] for idx in row], prediction))

[['plane', 'take', 'off', 'from', 'dallas', '<end>', 'from'],
 ['plane', 'land', 'at', 'dallas', '<end>', 'from', 'from'],
 ['plane', 'take', 'off', 'from', 'chicago', '<end>', 'from'],
 ['plane', 'land', 'at', 'chicago', '<end>', 'from', 'from'],
 ['plane', 'take', 'off', 'from', 'los', 'angeles', '<end>'],
 ['plane', 'land', 'at', 'los', 'angeles', '<end>', 'from'],
 ['plane', 'take', 'off', 'from', 'new', 'york', '<end>'],
 ['plane', 'land', 'at', 'new', 'york', '<end>', 'from'],
 ['plane', 'take', 'off', 'from', 'miami', '<end>', 'from'],
 ['plane', 'land', 'at', 'miami', '<end>', 'from', 'from']]